In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from utils.preprocess import *
from utils.model import *

### Extract ts features

In [14]:
train_data = read_data('train')

In [15]:
X_train = preprocess_data(train_data)

In [16]:
test_data = read_data('test')

In [17]:
X_test = preprocess_data(test_data)

In [18]:
cluster_train = pd.read_csv(config['data_path'] + 'cluster_train.csv')
X_train = X_train.merge(cluster_train, on='NPLV')

cluster_test = pd.read_csv(config['data_path'] + 'cluster_test.csv')
X_test = X_test.merge(cluster_test, on='NPLV')

In [19]:
chronom_data_train = pd.read_csv('data_chronom/chronom_duration_train_new3.csv')
chronom_data_test = pd.read_csv('data_chronom/chronom_duration_test_new3.csv')
preprocessed_plavki_train = pd.read_csv(config['data_path'] + 'preprocessed_plavki_train.csv')
preprocessed_plavki_test = pd.read_csv(config['data_path'] + 'preprocessed_plavki_test.csv')

In [20]:
X_train = X_train.merge(preprocessed_plavki_train, on='NPLV')
X_test  = X_test.merge(preprocessed_plavki_test, on='NPLV')
X_train = X_train.merge(chronom_data_train, on='NPLV')
X_test  = X_test.merge(chronom_data_test, on='NPLV')

In [21]:
X_train.to_csv(config['data_path'] + 'preprocessed_train.csv')
X_test.to_csv(config['data_path'] + 'preprocessed_test.csv')